<a href="https://colab.research.google.com/github/rmdnps10/23-1-Nlp-Project/blob/main/Project_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트 Pipeline

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')
nltk.download('wordnet')


In [ ]:
pip install transformers

In [ ]:
pip install xformers

## 0. 수민이가 크롤링한 텍스트 전처리
- 현대, 기아도 여기다 넣으면돼요

In [ ]:
csv_file_path = 'Hyundai_reddit_nogada.csv'

In [ ]:
import csv
samsung_list = []
with open(csv_file_path, 'r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        row_string = ','.join(row)
        samsung_list.append(row_string)

print(samsung_list)


In [ ]:
import re

def remove_special_characters(text):
    clean = re.sub('[^a-zA-Z0-9\s]', '', text)
    return clean

clean_list= []

for text in samsung_list:
  clean = remove_special_characters(text)
  clean_list.append(clean)

print(clean_list)


In [ ]:
def unify_text(text):
    unify = text.lower()
    return unify

unified_text = []

for text in samsung_list:
  unify = unify_text(text)
  unified_text.append(unify)

print(unified_text)


## 1-1. 키워드 추출 -> Term-Frequency

- 가장 많이 나온 명사 TOP50

In [ ]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from collections import Counter

def find_most_frequent_nouns(string_list, top_n=20):
    words = [word for string in string_list for word in word_tokenize(string)]
    tagged_words = pos_tag(words)
    nouns = [word for word, pos in tagged_words if pos.startswith('NN')]
    noun_counts = Counter(nouns)
    most_common_nouns = noun_counts.most_common(top_n)
    return most_common_nouns

most_frequent_nouns = find_most_frequent_nouns(unified_text, top_n=50)
for noun, count in most_frequent_nouns:
    print(noun, count)

- word cloud 생성 (코퍼스에서 명사만 추출해서 단어빈도수에 따라 클라우드 구성))

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# 명사 추출 함수
def extract_nouns(text):
    tokens = word_tokenize(text)
    nouns = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    return nouns

# 텍스트 데이터셋에서 명사 추출
nouns_list = []
for text in unified_text:
    nouns = extract_nouns(text)
    nouns_list.extend(nouns)

# 명사 리스트를 공백으로 연결하여 하나의 문자열로 만듦
nouns_text = ' '.join(nouns_list)

# WordCloud 생성
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(nouns_text)

# WordCloud 시각화
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


삼성 워드 클라우드 다운로드 코드

In [ ]:
wordcloud.to_file('hyundai_wordcloud.png')

## 1-2. 키워드 추출 -> Top2vec

In [ ]:
pip install top2vec

In [ ]:
from top2vec import Top2Vec

model = Top2Vec(samsung_list)

In [ ]:
 model.get_num_topics()

In [ ]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["hyundai"], num_topics=2)
for topic in topic_nums:
    model.generate_topic_wordcloud(topic)


##2. 주요 키워드에 대한 Sentiment Analysis



- 문장에 대하여 sentiment analysis을 통하여 감정 점수를 부여하여,일정 임계점 이상의 감정 점수가 부여된 문장만 필터링

In [ ]:
from nltk.corpus import sentiwordnet as swn

# 문자열 리스트

# 감정적인 단어가 있는지 확인하여 삭제할 문자열을 담을 리스트
filtered_list = []

# 각 문자열에 대해 감정적인 단어 여부를 확인하고 삭제할 문자열을 filtered_list에 추가
for sentence in unified_text:
    words = sentence.split()  # 문자열을 단어로 분리

    # 각 단어에 대해 감정적인 단어 여부 확인
    has_emotional_word = False
    for word in words:
        synsets = list(swn.senti_synsets(word))
        if synsets:
            has_emotional_word = True
            break

    if has_emotional_word:
        filtered_list.append(sentence)

print(filtered_list)
len(filtered_list)



 삼성과 관련된 주요 키워드를 포함하고 있는 문장만 필터링 ( 광고성, 아예 주제랑 벗어나는 말 제외하기 위함))

In [ ]:
# 문자열 리스트


# 원하는 키워드 리스트
##desired_keywords = ["samsung", "galaxy","s23","s22","s23u","ultra","apple","pay","oneui","s21","s22u","s23u","ultra", "screen", "camera", "battery","service"]
desired_keywords= ["hyundai", "sonata","elantra", "car", "engine", "dealer","oil","dealership", "service", "warranty"]
# 원하는 키워드가 없는 경우 리스트 요소를 제거
keyword_list = [sentence for sentence in filtered_list if any(keyword in sentence for keyword in desired_keywords)]

print(keyword_list)
len(keyword_list)


2차 필터링 후 문장 단위로 리스트를 만들어주기.

In [ ]:
def index_sentences(string_list):
    indexed_list = []
    for string in string_list:
        sentence_list = []
        temp_sentence = ''
        for char in string:
            temp_sentence += char
            if char in ['.', '?', '!']:
                sentence_list.append(temp_sentence.strip())
                temp_sentence = ''
        if temp_sentence:
            sentence_list.append(temp_sentence.strip())
        indexed_list.extend(sentence_list)
    return indexed_list


indexed_list = index_sentences(keyword_list)
print(indexed_list)
len(indexed_list)

**3차 필터링: "배터리"** 가 포함된 문장만 뽑아내서, 본격적인 분석

## **여기서부터 아래 코드의 경우, desired_keywords만 바꿔줘서 반복하여 코드돌리면됨**

- 키워드에 대한 긍/부정 csv 파일 다운로드
- 키워드에 대한 긍/부정 wordcloud 다운로드

<삼성의 경우>

Desired keywords =
1. ["s23","flip","fold","s22","note"]
2. ["battery"]
3. ["camera"]
4. ["software","oneui"]
5. ["screen"]


<현대의 경우>

Desired keywords =
1. ["sonata"]
2. ["elantra"]
3. ["engine"]
4. ["oil"]
4. ["dealer","customer service"]
5. ["design"]

In [ ]:
# 원하는 키워드 입력
desired_keywords = ["design"]

- 키워드 포함된 문장 모두 뽑아냅니다!

In [ ]:
battery_list = [sentence for sentence in indexed_list if any(keyword in sentence for keyword in desired_keywords)]


print(battery_list)
print(len(battery_list))

- Huggin Face의 sentiment analysis model 이용해서 긍/부정 판별

In [ ]:
from transformers import pipeline

# 감정 분석 모델 로드
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_analyzer = pipeline("sentiment-analysis", model=model_name)

# 분류할 문장들

sentences= battery_list

# 문장 분류
positive_list = []
negative_list = []

for sentence in sentences:
    result = sentiment_analyzer(sentence)[0]
    label = result['label']

    if label == "POSITIVE":
        positive_list.append(sentence)
    elif label == "NEGATIVE":
        negative_list.append(sentence)

# 결과 출력
print("Positive List:", positive_list)
print("Negative List:", negative_list)


 ### 키워드를 포함하고 있는 문장에 대한 긍정, 부정의 비율을 통한 점수 주고 키워드의 긍정/부정 코퍼스 csv파일로 다운로드 하는 코드
 -> 이거 일일이 보면서 인사이트 추출

In [ ]:

print('score: {0:.2f}'.format(len(positive_list)/ (len(positive_list) + len(negative_list)) *100) )
import csv
csv_file_path = '_'.join(desired_keywords) + "_positive.csv"
with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for item in positive_list:
        writer.writerow([item])
csv_file_path = '_'.join(desired_keywords) + "_negative.csv"
with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for item in negative_list:
        writer.writerow([item])

print(len(positive_list))
print(len(negative_list))
print("CSV 파일이 생성되었습니다.")

### 긍정적인 문장들만 따로 모아서 wordcloud생성

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# positive_list의 요소들을 공백으로 연결하여 하나의 문자열로 만듦
text = ' '.join(positive_list)

# WordCloud 생성
wordcloud = WordCloud(width=800, height=400, background_color='black').generate(text)

# WordCloud를 시각화
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()
image_file_name = '_'.join(desired_keywords) + '_positive.png'
wordcloud.to_file(image_file_name)
print("이미지 파일이 생성되었습니다.")


### 부정적인 문장의 wordcloud

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

text = ' '.join(negative_list)

# WordCloud 생성
wordcloud = WordCloud(width=800, height=400, background_color='black').generate(text)

# WordCloud를 시각화
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()
image_file_name = '_'.join(desired_keywords) + '_negative.png'
wordcloud.to_file(image_file_name)
print("이미지 파일이 생성되었습니다.")
